# **Space  X Falcon 9 First Stage Landing Prediction**

## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)

<p>Sucess landing:</p>

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


<p>Unsuccessful landing:</p>

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)

More specifically, the launch records are stored in a HTML table 

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


---

  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

Some helper functions for the process web scraped HTML table

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML table cell
    Input: the element of a table date cell extract exstra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0: 2]

def booster_version(table_cells):
    """
    Tihs function returns the booster version from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strngs) if i%2 == 0][0: -1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the element of a table data cell extracts extra row
    """
    out = [i for i in table_cells.strings][0]

def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

### Request the Falcon 9 Launch Wiki page

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches"

response = requests.get(url)

if(response.status_code == 200):
    soup = BeautifulSoup(response.content, 'html.parser')
    print("Page title: ", soup.title.text)
else:
    print(f"Error, status code: {response.status_code}")

Page title:  List of Falcon 9 and Falcon Heavy launches - Wikipedia


### Extracting all column/variable name from the HTML table hader 

Find all tables on the wiki page

In [5]:
html_tables = soup.find_all('table')
html_tables

[<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody><tr class="is-sticky">
 <th scope="col">Flight No.
 </th>
 <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
 </th>
 <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_21-0"><a href="#cite_note-booster-21">[c]</a></sup>
 </th>
 <th scope="col">Launch<br/>site
 </th>
 <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_22-0"><a href="#cite_note-Dragon-22">[d]</a></sup>
 </th>
 <th scope="col">Payload mass
 </th>
 <th scope="col">Orbit
 </th>
 <th scope="col">Customer
 </th>
 <th scope="col">Launch<br/>outcome
 </th>
 <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
 </th></tr>
 <tr id="F9-078">
 <th rowspa

Starting from third table is our target table contains the actual launch records

In [6]:
first_launch_table = html_tables[2] #2022
first_launch_table

<table class="wikitable plainrowheaders collapsible" id="2022ytd" style="width: 100%;">
<tbody><tr class="is-sticky">
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_21-2"><a href="#cite_note-booster-21">[c]</a></sup>
</th>
<th scope="col">Launch<br/>site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_22-2"><a href="#cite_note-Dragon-22">[d]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr id="F9-135">
<th rowspan="2" scope

Iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one

In [7]:
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


table_headers = first_launch_table.find_all('th')

column_names = []

for i in table_headers:
    i = extract_column_from_header(i)
    if((i != None) and (len(i) > 0)):
        column_names.append(i)
    else:
        pass

column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'FH 4']

## Creating a data frame by parsing the launch HTML tables

Empty dictionary with keys from the extracted coolumn names in the previous task. Late, his dictionarywill be converted into a Pandas dataframe

In [12]:
launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column
del launch_dict['Date and time ( )']

# Initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
#Added some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [13]:
launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch outcome': [],
 'FH 4': None,
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

Next, fill up the `launch_dict` with records extracted from table rows

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises such as reference links `B0004.1[8]`, missing values `N-A [e]`, inconsistent formatting, etc